In [1]:
# Import libraries

%pylab inline

import numpy as np
import scipy.io
from astropy.io import ascii
from astropy.table import Table, Column
from astropy.io import fits
import matplotlib.pyplot as plt
import glob
import pickle
import time
import seaborn as sns
sns.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Construct likelihood function

#========================================================================
#This function calculates the likelihood function for a given
#model and photometry dataset. And returns the lilekihood and chi^2.

#CALLING SEQUENCE
# > likelihood(obs_fluxes_resolved, obs_errors_resolved, observed_fluxes_unresolved, 
#                   observed_errors_unresolved, model_fluxes, weight))
# obs_fluxes_resolved: An array of dimension m x n where m is the number of sources and n is
# the number of resolved bands. This array contains the resolved fluxes.
#
# obs_errors_resolved: The associated errors to resolved observations. Same dimension.
#
# observed_fluxes_unresolved: An array of dimension p where p is the number of unresolved
# bands. It contains the unresolved fluxes.
#
# model_fluxes: the fluxes of the models to comapred with data. The array must have m rows
# and n+p columns.
#
# weight: The weight to be given to each datapoint (uniform, etc.)
#========================================================================

def likelihood(obs_fluxes_resolved, obs_errors_resolved, obs_fluxes_unresolved, 
               obs_errors_unresolved, model_fluxes, weight):
    
    # Get number of sources
    n_sources = len(obs_fluxes_resolved)
    # Get number of bands in which sources are resolved
    n_bands_resolved = len(obs_fluxes_resolved[0])
    n_data = len(model_fluxes[0])
        
    # Remove no-data points
    zeros_index = np.array(where(obs_fluxes_unresolved == 0.0))
    obs_fluxes_uresolved = np.delete(obs_fluxes_unresolved,zeros_index)
    obs_errors_unresolved = np.delete(obs_errors_unresolved,zeros_index)
    for source in arange(n_sources): model_fluxes[source] = np.delete(model_fluxes[source],n_bands_resolved+zeros_index)
    
    for source in arange(n_sources):
        zeros_index = np.array(where(obs_fluxes_resolved[source] == 0.0))
        obs_fluxes_resolved[source] = np.delete(obs_fluxes_resolved[source],zeros_index)
        obs_errors_resolved[source] = np.delete(obs_errors_resolved[source],zeros_index)
        model_fluxes[source] = np.delete(model_fluxes[source],zeros_index)
       
    #print obs_fluxes_resolved,obs_fluxes_unresolved,model_fluxes
    
    chi_squared = 0
    
    # Calculate chi squared for resolved bands
    
    for source in arange(n_sources):
        
        # Get upper limits
        ind_upper_res = np.where(obs_errors_resolved[source] == -1)
        
        # Likelihood is 0 if model flux at upper limit band is larger than the measured one.
        if (model_fluxes[source][ind_upper_res] > obs_fluxes_resolved[source][ind_upper_res]):
            chi_squared += float('Inf')
            continue
        
        # Updated number of resolved datapoints
        n_data_res = len(obs_fluxes_resolved[source])
    
        # Unbiased flux in log space (since we are fitting in logarithmic space, see Robitaille et al. 2007)
        log_obs_flux = log10(obs_fluxes_resolved[source]) \
        -0.5*(1.0/log(10.))*(1.0/(obs_fluxes_resolved[source]**2.0)) \
        *obs_errors_resolved[source]*obs_errors_resolved[source]
        
        # Unbiased variances in log space (since we are fitting in logarithmic space, see Robitaille et al. 2007)
        log_obs_var2 = (((1.0/log(10.))*(1.0/obs_fluxes_resolved[source]))**2.0) \
        *obs_errors_resolved[source]*obs_errors_resolved[source] 
    
        # Model flux in log space
        log_mod_flux = log10(model_fluxes[source][0:n_data_res]) 
    
        # Reduced chi^2
        #print source, n_data_res, (1.0/n_data_res)*sum(((log_obs_flux - log_mod_flux)*(log_obs_flux - log_mod_flux)) \
                                            #/(2.0*weight*(log_obs_var2)))
        chi_squared += (1.0/n_data_res)*sum(((log_obs_flux - log_mod_flux)*(log_obs_flux - log_mod_flux)) \
                                            /(2.0*weight*(log_obs_var2)))
    
    # Calculate chi squared for unresolved bands
    
    # Get upper limits
    ind_upper_unres = np.where(obs_errors_unresolved == -1)
    
    # Likelihood is 0 if model flux at upper limit band is larger than the measured one.
    mod_flux_upper = 0.0
    for source in arange(n_sources): 
        mod_flux_upper += model_fluxes[source][n_data_res:][ind_upper_unres]
    if (mod_flux_upper > obs_fluxes_unresolved[ind_upper_unres]):
        chi_squared += float('Inf')
    
    # Updated number of unresolved datapoints
    n_data_unres = len(obs_fluxes_unresolved)
    
    # Unbiased flux in log space (since we are fitting in logarithmic space, see Robitaille et al. 2007)
    log_obs_flux = log10(obs_fluxes_unresolved)-0.5*(1.0/log(10.)) \
    *(1.0/(obs_fluxes_unresolved**2.0))*obs_errors_unresolved*obs_errors_unresolved
        
    # Unbiased variances in log space (since we are fitting in logarithmic space, see Robitaille et al. 2007)
    log_obs_var2 = (((1.0/log(10.))*(1.0/obs_fluxes_unresolved))**2.0) \
    *obs_errors_unresolved*obs_errors_unresolved 
    
    # Model flux in log space
    mod_flux = np.zeros(n_data_unres)
    for source in arange(n_sources): 
        mod_flux += model_fluxes[source][n_data_res:]
    log_mod_flux = log10(mod_flux)
        
        
    # Reduced chi^2
    #print 'unres', n_data_unres, (1.0/n_data_unres)*sum(((log_obs_flux - log_mod_flux)*(log_obs_flux - log_mod_flux)) \
                                        #/(2.0*weight*(log_obs_var2)))
    chi_squared += (1.0/n_data_unres)*sum(((log_obs_flux - log_mod_flux)*(log_obs_flux - log_mod_flux)) \
                                        /(2.0*weight*(log_obs_var2)))
            
    # Likelihood probability (assuming Gaussian errors). Note that the chi squared value is the sum
    # of both resolved and unresolved bands
    if (chi_squared <= 745):
        likelihood = exp(-chi_squared)
    else:
        likelihood = np.exp(-745)
    
    likelihood_dict = {'chi2':chi_squared, 'likelihood':likelihood}
    
    #print 'chisq_tot', chi_squared
    #print 'prob_tot', likelihood
    
    return likelihood_dict   

In [4]:
# Define distance function (on the mass-age plane)
def distance(age_old,mass_old,age_new,mass_new):
    return sqrt((log10(age_new)-log10(age_old))**2+(log10(mass_new)-log10(mass_old))**2)

In [9]:
# Define potential energy function, which is basically the chi^2 value
# theta is a vector with the parameters m*, t*, inclination, av, and distance
# Need function that takes ID and gives m* and t* (already have it)
def U(theta,obs_fluxes_resolved, obs_errors_resolved, 
      obs_fluxes_unresolved, obs_errors_unresolved):
         
    source_fluxes = []
    # Loop over bands to assign them fluxes
    for filter in filter_names:
        nombres = idl_dict1[filter]['name'][0]
        angulos = idl_dict1[filter]['angle'][0]
        at = where((nombres == theta[0]) & (angulos == idl_dict1['model_str_bu']['angle'][0][theta[1]])) 
        source_fluxes.append(mean(idl_dict1[filter]['fluxes'][0][49][at]))

    source_fluxes = np.array(source_fluxes)

    model_fluxes = 10.**(log10(source_fluxes)) 
    # Apply extinction and distance scaling
    model_fluxes *= ext_law(filter_wavelengths,theta[2],wavita,opacita)/(theta[3]**2)   
    chi2 = likelihood(obs_fluxes_resolved,obs_errors_resolved,obs_fluxes_unresolved,
    obs_errors_unresolved,model_fluxes,1.0)['chi2']
    -(log(prior_t_0)+log(prior_incl_0))
    
    return chi2
    

In [ ]:
# Define gradient of potential energy
def dU(theta,obs_fluxes_resolved, obs_errors_resolved, 
      obs_fluxes_unresolved, obs_errors_unresolved):
    
    
    